In [ ]:
# ============================================
# 1. Install Dependencies & Build OpenFace
# ============================================
!apt-get update -qq
!apt-get install -y build-essential cmake git wget unzip \
    libopenblas-dev liblapack-dev libatlas-base-dev gfortran \
    libboost-all-dev libopencv-dev python3-opencv \
    libtbb-dev libeigen3-dev

# ---- Install dlib (C++ library, needed for OpenFace) ----
!git clone https://github.com/davisking/dlib.git
%cd dlib
!mkdir build
%cd build
!cmake ..
!cmake --build . --config Release
!make install
%cd /content

# ---- Building OpenFace ----
!git clone https://github.com/TadasBaltrusaitis/OpenFace.git
%cd OpenFace
!mkdir build
%cd build
!cmake ..
!make -j4

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
build-essential is already the newest version (12.9ubuntu3).
gfortran is already the newest version (4:11.2.0-1ubuntu1).
liblapack-dev is already the newest version (3.10.0-2ubuntu1).
libatlas-base-dev is already the newest version (3.10.3-12ubuntu1).
libboost-all-dev is already the newest version (1.74.0.3ubuntu7).
libopenblas-dev is already the newest version (0.3.20+ds-1).
libtbb-dev is already the newest version (2021.5.0-7ubuntu2).
libtbb-dev set to manually installed.
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
git is already the newest version (1:2.34.1-1ubuntu1.15).
unzip is already the newest version (6.0-26ubuntu3.2).
wget is already the newest version (1.21.2-2ubuntu1.1).
lib

In [ ]:
# move to root of OpenFace (where download_models.sh should be)
%cd /content/OpenFace

# make script executable if needed
!chmod +x download_models.sh

# Run it
!bash download_models.sh

# If that fails or is incomplete, fallback — manually fetch patch experts from Dropbox
!wget -O patch_experts.zip "https://www.dropbox.com/sh/o8g1530jle17spa/AADRntSHl_jLInmrmSwsX-Qsa?dl=1"
!unzip -o patch_experts.zip -d build/bin/model/patch_experts

# Also fetch Haar cascade if missing
!wget -O build/bin/model/haarcascade_frontalface_alt.xml "https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt.xml"

/content/OpenFace
--2025-09-30 11:10:20--  https://www.dropbox.com/s/7na5qsjzz8yfoer/cen_patches_0.25_of.dat
Resolving www.dropbox.com (www.dropbox.com)... 162.125.4.18, 2620:100:6016:18::a27d:112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.4.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/nbdylr41jbtpykft3m56s/cen_patches_0.25_of.dat?rlkey=r9juzsg39pn40wvv0seyfddz4 [following]
--2025-09-30 11:10:20--  https://www.dropbox.com/scl/fi/nbdylr41jbtpykft3m56s/cen_patches_0.25_of.dat?rlkey=r9juzsg39pn40wvv0seyfddz4
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucb5cc2c330fada7bf33c1815299.dl.dropboxusercontent.com/cd/0/inline/CyVhn0cAY0QHehOY9KOkKw9GDjboNYbYIGzHVSvBo14VDGj9Xaje53bGiqfON2wqleJ2eqtGgrn1l0i86E-fq8RopUDpE6rw7yf3d7_kveObfjQtIq5hoem8LzpWMz0cdGo/file# [following]
--2025-09-30 11:10:21--  https://ucb5cc2c330fada7bf33c1815299.dl.dropbo

In [ ]:
openface_bin = "/content/OpenFace/build/bin/FeatureExtraction"

In [ ]:
# ============================================
# 2. Setup Directories
# ============================================
import os

input_dir = "/content/videos"    # <-- Upload your videos here
output_dir = "/content/output"   # <-- Results will be saved here

folders = ["csv", "txt", "hog", "avi", "aligned"]
for folder in folders:
    os.makedirs(os.path.join(output_dir, folder), exist_ok=True)

In [ ]:
# ============================================
# 3. Upload Videos
# ============================================
from google.colab import files

print("📤 Upload your video files (mp4, avi, mov, mkv)...")
uploaded = files.upload()

os.makedirs(input_dir, exist_ok=True)
for fname in uploaded.keys():
    os.rename(fname, os.path.join(input_dir, fname))

📤 Upload your video files (mp4, avi, mov, mkv)...


Saving testsub.mp4 to testsub.mp4


In [ ]:
# ============================================
# 4. Process All Videos in Input Folder
# ============================================
import shutil
import subprocess
import time
from tqdm import tqdm

valid_exts = (".mp4", ".avi", ".mov", ".mkv")
video_list = [f for f in os.listdir(input_dir) if f.lower().endswith(valid_exts)]

pbar = tqdm(video_list, desc="Processing videos", unit="video")
start_time = time.time()

for idx, video_name in enumerate(pbar):
    video_path = os.path.normpath(os.path.join(input_dir, video_name))

    if os.path.exists(video_path):
        # Temporary output folder
        temp_out = os.path.join(output_dir, f"temp_{os.path.splitext(video_name)[0]}")
        os.makedirs(temp_out, exist_ok=True)

        # Run OpenFace
        subprocess.run([
            openface_bin,
            "-f", video_path,
            "-out_dir", temp_out
        ])

        # Move files to their respective folders
        for file in os.listdir(temp_out):
            src = os.path.join(temp_out, file)
            if file.endswith(".csv"):
                shutil.move(src, os.path.join(output_dir, "csv", file))
            elif file.endswith(".txt"):
                shutil.move(src, os.path.join(output_dir, "txt", file))
            elif file.endswith(".hog"):
                shutil.move(src, os.path.join(output_dir, "hog", file))
            elif file.endswith(".avi"):
                shutil.move(src, os.path.join(output_dir, "avi", file))
            elif os.path.isdir(src) and file.endswith("_aligned"):
                shutil.move(src, os.path.join(output_dir, "aligned", file))

        shutil.rmtree(temp_out)

        # ETA
        elapsed = time.time() - start_time
        avg_time_per_video = elapsed / (idx + 1)
        remaining_videos = len(video_list) - (idx + 1)
        eta = remaining_videos * avg_time_per_video
        pbar.set_description(f"Processing {video_name} | ETA: {eta/60:.1f} min")

    else:
        tqdm.write(f"⏭ Skipping {video_name} (not found)")

Processing testsub.mp4 | ETA: 0.0 min: 100%|██████████| 1/1 [02:43<00:00, 163.45s/video]


In [ ]:
# ============================================
# 5. Download Results
# ============================================
from google.colab import files
import shutil

# Zip results for download
shutil.make_archive("/content/openface_results", 'zip', output_dir)
files.download("/content/openface_results.zip")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>